In [1]:
import snowflake.connector

from snowduck import start_patch_snowflake

## 🔄 Persistence: In-Memory vs File-Based Storage

SnowDuck supports two storage modes:

**1. In-Memory (Default)**: Fast, isolated, data lost on exit
```python
start_patch_snowflake()  # or start_patch_snowflake(db_file=':memory:')
```

**2. File-Based**: Persistent, data survives restarts, perfect for testing
```python
start_patch_snowflake(db_file='my_test_data.duckdb')
```

**3. Fresh Start**: Reset clears existing data (great for notebooks!)
```python
start_patch_snowflake(db_file='demo.duckdb', reset=True)
```

This notebook uses **file-based storage with reset=True** for a clean start every time!

In [2]:
# Enable persistence with file-based storage
# Use ':memory:' for in-memory (default) or a file path for persistence
# reset=True gives you a fresh start (useful for notebooks!)
start_patch_snowflake(db_file="snowduck_examples.duckdb", reset=True)

# ❄️🦆 SnowDuck Examples

This notebook demonstrates SnowDuck's features - a Snowflake-compatible SQL engine powered by DuckDB.

**💡 Important**: Run cells from top to bottom in order! Some cells depend on previous cells.

## Table of Contents
1. [Basic Queries](#1-basic-queries)
2. [Database & Schema Management](#2-database--schema-management)
3. [Table Operations & Data Seeding](#3-table-operations--data-seeding)
4. [Conditional Logic & Expressions](#4-conditional-logic--expressions)
5. [Advanced Queries (CTEs & Joins)](#5-advanced-queries-ctes--joins)
6. [Information Schema](#6-information-schema)
7. [Variables & Session Management](#7-variables--session-management)
8. [Window Functions & Analytics](#8-window-functions--analytics)
9. [Aggregate Functions](#9-aggregate-functions)
10. [String Functions](#10-string-functions)

---

## 1. Basic Queries

In [3]:
# Simple SELECT query
with snowflake.connector.connect() as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT 'Hello, SnowDuck!' as message")
    result = cursor.fetchone()
    print(f"Message: {result[0]}")
    cursor.close()

Message: Hello, SnowDuck!


In [4]:
# Multiple columns and calculations
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    cursor.execute("""
        SELECT 
            1 + 1 as addition,
            10 * 5 as multiplication,
            'Snow' || 'Duck' as concatenation
    """)
    result = cursor.fetchone()
    print(
        f"Addition: {result[0]}, Multiplication: {result[1]}, Concatenation: {result[2]}"
    )

Addition: 2, Multiplication: 50, Concatenation: SnowDuck


## 2. Database & Schema Management

In [5]:
# Create databases and schemas
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    # Create databases
    cursor.execute("CREATE DATABASE IF NOT EXISTS my_database")
    cursor.execute("CREATE DATABASE IF NOT EXISTS analytics_db")

    # Show all databases
    cursor.execute("SHOW DATABASES")
    databases = cursor.fetchall()
    print("Databases:", [db[1] for db in databases])

Databases: ['analytics_db', 'my_database', 'snowduck_examples', 'system', 'temp']


In [6]:
# Use database and create schemas
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    cursor.execute("USE DATABASE my_database")
    cursor.execute("CREATE SCHEMA IF NOT EXISTS public")
    cursor.execute("CREATE SCHEMA IF NOT EXISTS staging")

    cursor.execute("SHOW SCHEMAS")
    schemas = cursor.fetchall()
    print("Schemas:", [s[1] for s in schemas])

Schemas: []


## 3. Table Operations & Data Seeding

**SnowDuck provides first-class data seeding with `seed_table()`:**

```python
from snowduck import seed_table

# Dict of lists (easiest!)
seed_table(conn, 'users', {
    'id': [1, 2, 3],
    'name': ['Alice', 'Bob', 'Carol']
})

# Or pandas DataFrame
seed_table(conn, 'products', df)
```

Perfect for test fixtures, demos, and development!

In [7]:
# ⭐ Best Practice: Use seed_table() for easy test data!
import pandas as pd

from snowduck import seed_table

# Create test data - your choice of format!
employees_data = {
    "id": [1, 2, 3, 4, 5, 6],
    "name": [
        "Alice Johnson",
        "Bob Smith",
        "Carol White",
        "David Brown",
        "Eve Davis",
        "Frank Wilson",
    ],
    "department": [
        "Engineering",
        "Sales",
        "Engineering",
        "Marketing",
        "Engineering",
        "Sales",
    ],
    "salary": [95000, 75000, 105000, 68000, 98000, 82000],
    "hire_date": pd.to_datetime(
        [
            "2020-01-15",
            "2019-03-22",
            "2018-07-10",
            "2021-09-01",
            "2020-11-20",
            "2019-05-14",
        ]
    ),
}

with snowflake.connector.connect() as conn:
    cursor = conn.cursor()
    cursor.execute("USE DATABASE my_database")
    cursor.execute("USE SCHEMA public")

    # One line to seed the table! 🎉
    rows = seed_table(conn, "employees", employees_data)
    print(f"✅ Seeded {rows} employees using seed_table()")

    # Show first 3 employees
    cursor.execute("SELECT * FROM employees LIMIT 3")
    results = cursor.fetchall()
    print("\nFirst 3 employees:")
    for row in results:
        print(f"  {row[1]} - {row[2]} - ${row[3]:,}")

✅ Seeded 6 employees using seed_table()

First 3 employees:
  Alice Johnson - Engineering - $95,000
  Bob Smith - Sales - $75,000
  Carol White - Engineering - $105,000


In [8]:
# Alternative: Load from files (DuckDB's superpower!)
# DuckDB can read CSV, Parquet, JSON directly
import pandas as pd

with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    # Example: Read from Parquet (if you have test data files)
    # cursor.execute("CREATE TABLE sales AS SELECT * FROM 'test_data/sales.parquet'")

    # Example: Read from CSV
    # cursor.execute("CREATE TABLE customers AS SELECT * FROM 'test_data/customers.csv'")

    # For this demo, we'll create a temp parquet file from our data
    df = pd.DataFrame(employees_data)
    df.to_parquet("/tmp/employees.parquet", index=False)

    cursor.execute("""
        CREATE OR REPLACE TABLE employees_from_file AS 
        SELECT * FROM '/tmp/employees.parquet'
    """)

    cursor.execute("SELECT COUNT(*) FROM employees_from_file")
    count = cursor.fetchone()
    print(f"✅ Loaded {count[0]} employees from Parquet file")

✅ Loaded 6 employees from Parquet file


In [9]:
# Alternative Method 2: CREATE TABLE AS with inline data
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    cursor.execute("""
        CREATE OR REPLACE TABLE departments AS
        SELECT * FROM (VALUES
            ('Engineering', 3, 99333),
            ('Sales', 2, 78500),
            ('Marketing', 1, 68000)
        ) AS t(name, employee_count, avg_salary)
    """)

    cursor.execute("SELECT * FROM departments")
    results = cursor.fetchall()
    print("Departments:")
    for row in results:
        print(f"  {row[0]}: {row[1]} employees, avg ${row[2]:,}")

Departments:
  Engineering: 3 employees, avg $99,333
  Sales: 2 employees, avg $78,500
  Marketing: 1 employees, avg $68,000


## 4. Conditional Logic & Expressions

In [10]:
# Subquery with CASE statements
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    cursor.execute("""
        SELECT 
            name,
            department,
            salary,
            CASE 
                WHEN salary >= 100000 THEN 'Senior'
                WHEN salary >= 80000 THEN 'Mid-Level'
                ELSE 'Junior'
            END as level,
            CASE 
                WHEN hire_date < '2020-01-01' THEN 'Veteran'
                ELSE 'Recent'
            END as tenure
        FROM employees
        ORDER BY salary DESC
    """)

    results = cursor.fetchall()
    print("Employee Classification:")
    print("-" * 70)
    print(f"{'Name':<15} {'Department':<12} {'Salary':>10} {'Level':<10} {'Tenure':<8}")
    print("-" * 70)
    for row in results:
        print(f"{row[0]:<15} {row[1]:<12} ${row[2]:>9,} {row[3]:<10} {row[4]:<8}")

Employee Classification:
----------------------------------------------------------------------
Name            Department       Salary Level      Tenure  
----------------------------------------------------------------------
Carol White     Engineering  $  105,000 Senior     Veteran 
Eve Davis       Engineering  $   98,000 Mid-Level  Recent  
Alice Johnson   Engineering  $   95,000 Mid-Level  Recent  
Frank Wilson    Sales        $   82,000 Mid-Level  Veteran 
Bob Smith       Sales        $   75,000 Junior     Veteran 
David Brown     Marketing    $   68,000 Junior     Recent  


## 5. Advanced Queries (CTEs & Joins)

In [11]:
# CTE (Common Table Expression) with multiple aggregations
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    cursor.execute("""
        WITH dept_stats AS (
            SELECT 
                department,
                COUNT(*) as emp_count,
                AVG(salary) as avg_salary
            FROM employees
            GROUP BY department
        ),
        high_performers AS (
            SELECT 
                e.name,
                e.department,
                e.salary,
                ds.avg_salary
            FROM employees e
            JOIN dept_stats ds ON e.department = ds.department
            WHERE e.salary > ds.avg_salary
        )
        SELECT 
            name,
            department,
            salary,
            ROUND(avg_salary, 0) as dept_avg,
            ROUND((salary - avg_salary) / avg_salary * 100, 1) as pct_above_avg
        FROM high_performers
        ORDER BY pct_above_avg DESC
    """)

    results = cursor.fetchall()
    print("Above-Average Performers:")
    print("-" * 85)
    print(
        f"{'Name':<15} {'Department':<12} {'Salary':>10} {'Dept Avg':>10} {'% Above':>8}"
    )
    print("-" * 85)
    for row in results:
        print(
            f"{row[0]:<15} {row[1]:<12} ${row[2]:>9,} ${row[3]:>9,.0f} {row[4]:>7.1f}%"
        )

Above-Average Performers:
-------------------------------------------------------------------------------------
Name            Department       Salary   Dept Avg  % Above
-------------------------------------------------------------------------------------
Carol White     Engineering  $  105,000 $   99,333     5.7%
Frank Wilson    Sales        $   82,000 $   78,500     4.5%


## 6. Information Schema

In [12]:
# Get column information
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    cursor.execute("""
        SELECT column_name, data_type, is_nullable
        FROM information_schema.columns
        WHERE table_name = 'employees'
        ORDER BY ordinal_position
    """)

    results = cursor.fetchall()
    print("Employees table structure:")
    print("-" * 50)
    print(f"{'Column':<15} {'Type':<15} {'Nullable':<10}")
    print("-" * 50)
    for row in results:
        print(f"{row[0]:<15} {row[1]:<15} {row[2]:<10}")

Employees table structure:
--------------------------------------------------
Column          Type            Nullable  
--------------------------------------------------
id              INTEGER         YES       
name            VARCHAR         YES       
department      VARCHAR         YES       
salary          INTEGER         YES       
hire_date       TIMESTAMP       YES       


In [13]:
# List all tables in the database
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    cursor.execute("USE DATABASE my_database")
    cursor.execute("""
        SELECT table_catalog, table_schema, table_name, table_type
        FROM information_schema.tables
        WHERE table_schema NOT IN ('information_schema', 'pg_catalog')
        ORDER BY table_schema, table_name
    """)

    results = cursor.fetchall()
    print(f"Tables in my_database ({len(results)} total):")
    for row in results:
        print(f"  {row[0]}.{row[1]}.{row[2]} ({row[3]})")

Tables in my_database (8 total):
  _snowduck_account._information_schema._columns (VIEW)
  _snowduck_account._information_schema._columns_ext (BASE TABLE)
  _snowduck_account._information_schema._tables_ext (BASE TABLE)
  _snowduck_account._information_schema._users_ext (BASE TABLE)
  _snowduck_account._information_schema._warehouses_ext (BASE TABLE)
  snowduck_examples.main.departments (BASE TABLE)
  snowduck_examples.main.employees (BASE TABLE)
  snowduck_examples.main.employees_from_file (BASE TABLE)


## 7. Variables & Session Management

In [14]:
# Session variables - Set them first!
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    # Set variables
    cursor.execute("SET min_salary = 80000")
    cursor.execute("SET target_dept = 'Engineering'")
    print("✅ Set session variables: min_salary=80000, target_dept='Engineering'")

    # Use variables in queries
    cursor.execute("""
        SELECT name, salary, department
        FROM employees
        WHERE salary >= $min_salary
        AND department = $target_dept
        ORDER BY salary DESC
    """)

    results = cursor.fetchall()
    print("\nHigh-earning Engineers:")
    for row in results:
        print(f"  {row[0]}: ${row[1]:,}")

✅ Set session variables: min_salary=80000, target_dept='Engineering'

High-earning Engineers:
  Carol White: $105,000
  Eve Davis: $98,000
  Alice Johnson: $95,000


In [15]:
# Note: Session variables are connection-scoped, not global
# They need to be set again for each new connection
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    cursor.execute("SET test_var = 'Hello from new connection'")
    cursor.execute("SELECT $test_var")
    result = cursor.fetchone()
    print(f"✅ Variable in same connection: {result[0]}")

✅ Variable in same connection: Hello from new connection


## 8. Window Functions & Analytics

In [16]:
# Window functions for ranking and analytics
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    cursor.execute("""
        SELECT 
            name,
            department,
            salary,
            ROW_NUMBER() OVER (ORDER BY salary DESC) as overall_rank,
            RANK() OVER (PARTITION BY department ORDER BY salary DESC) as dept_rank,
            DENSE_RANK() OVER (ORDER BY salary DESC) as dense_rank
        FROM employees
        ORDER BY salary DESC
    """)

    results = cursor.fetchall()
    print("Employee Rankings:")
    print("-" * 80)
    print(
        f"{'Name':<15} {'Department':<12} {'Salary':>10} {'Overall':>8} {'Dept':>6} {'Dense':>6}"
    )
    print("-" * 80)
    for row in results:
        print(
            f"{row[0]:<15} {row[1]:<12} ${row[2]:>9,} {row[3]:>8} {row[4]:>6} {row[5]:>6}"
        )

Employee Rankings:
--------------------------------------------------------------------------------
Name            Department       Salary  Overall   Dept  Dense
--------------------------------------------------------------------------------
Carol White     Engineering  $  105,000        1      1      1
Eve Davis       Engineering  $   98,000        2      2      2
Alice Johnson   Engineering  $   95,000        3      3      3
Frank Wilson    Sales        $   82,000        4      1      4
Bob Smith       Sales        $   75,000        5      2      5
David Brown     Marketing    $   68,000        6      1      6


## 9. Aggregate Functions

In [17]:
# Aggregate functions with GROUP BY
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    cursor.execute("""
        SELECT 
            department,
            COUNT(*) as employee_count,
            AVG(salary) as avg_salary,
            MIN(salary) as min_salary,
            MAX(salary) as max_salary,
            SUM(salary) as total_payroll
        FROM employees
        GROUP BY department
        ORDER BY avg_salary DESC
    """)

    results = cursor.fetchall()
    print("Department Statistics:")
    print("-" * 80)
    for row in results:
        print(f"Department: {row[0]}")
        print(f"  Employees: {row[1]}")
        print(f"  Avg Salary: ${row[2]:,.2f}")
        print(f"  Salary Range: ${row[3]:,} - ${row[4]:,}")
        print(f"  Total Payroll: ${row[5]:,}")
        print()

Department Statistics:
--------------------------------------------------------------------------------
Department: Engineering
  Employees: 3
  Avg Salary: $99,333.33
  Salary Range: $95,000 - $105,000
  Total Payroll: $298,000

Department: Sales
  Employees: 2
  Avg Salary: $78,500.00
  Salary Range: $75,000 - $82,000
  Total Payroll: $157,000

Department: Marketing
  Employees: 1
  Avg Salary: $68,000.00
  Salary Range: $68,000 - $68,000
  Total Payroll: $68,000



## 10. String Functions

In [18]:
# String manipulation functions
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    cursor.execute("""
        SELECT 
            name,
            UPPER(name) as uppercase,
            LOWER(name) as lowercase,
            LENGTH(name) as name_length,
            CONCAT(name, ' - ', department) as full_info,
            SUBSTRING(name, 1, 5) as short_name
        FROM employees
        WHERE id <= 3
    """)

    results = cursor.fetchall()
    print("String Functions Demo:")
    for row in results:
        print(f"  Original: {row[0]}")
        print(f"  Upper: {row[1]}, Lower: {row[2]}, Length: {row[3]}")
        print(f"  Full Info: {row[4]}")
        print()

String Functions Demo:
  Original: Alice Johnson
  Upper: ALICE JOHNSON, Lower: alice johnson, Length: 13
  Full Info: Alice Johnson - Engineering

  Original: Bob Smith
  Upper: BOB SMITH, Lower: bob smith, Length: 9
  Full Info: Bob Smith - Sales

  Original: Carol White
  Upper: CAROL WHITE, Lower: carol white, Length: 11
  Full Info: Carol White - Engineering



---

## Summary

SnowDuck provides comprehensive Snowflake SQL compatibility including:

✅ **DDL Operations**: CREATE/DROP databases, schemas, tables  
✅ **DML Operations**: INSERT, SELECT with complex WHERE clauses  
✅ **String Functions**: UPPER, LOWER, CONCAT, LENGTH, SUBSTRING  
✅ **Aggregate Functions**: COUNT, SUM, AVG, MIN, MAX with GROUP BY  
✅ **Window Functions**: ROW_NUMBER, RANK, DENSE_RANK, PARTITION BY  
✅ **Session Variables**: SET/SELECT $variable syntax  
✅ **Information Schema**: Query metadata about databases, tables, columns  
✅ **Advanced SQL**: CTEs, subqueries, JOINs, CASE statements  

**Perfect for:**
- 🧪 Testing SQL logic locally
- 🚀 Rapid prototyping  
- 💰 Development without cloud costs
- 📊 CI/CD pipeline integration

See [README.md](../README.md) for installation and more information.